In [1]:
import os

In [33]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
import getpass
from langsmith.wrappers import wrap_openai
from langsmith import traceable


In [57]:
# client = wrap_openai(AzureChatOpenAI())

In [68]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_TRACING_API_KEY"] = getpass.getpass("Enter your API key: ")

In [69]:
load_dotenv()

True

In [70]:
@traceable
def model():
    model1 = AzureChatOpenAI(
        api_key="",
        azure_deployment="pstestopenaidply-saxbryqsm4zao",
        azure_endpoint= "https://pstestopenaidply-saxbryqsm4zao.openai.azure.com/",
        openai_api_version="2024-02-01",                            ## Find the api version in chat playground "view code"
    )
    return model1

In [71]:
messages = [
    SystemMessage(content="Hello! I am a chatbot. How can I help you today?"),
    HumanMessage(content="Hi"),
]

In [72]:
model = model()
result = model.invoke(messages)
result

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 27, 'total_tokens': 36}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-2fb8df53-28f0-4e9a-b8e8-96fce13778a5-0', usage_metadata={'input_tokens': 27, 'output_tokens': 9, 'total_tokens': 36})

In [73]:
parser = StrOutputParser()

In [74]:
parser.invoke(result)

'Hello! How can I assist you today?'

In [75]:
chain = model | parser

In [76]:
chain.invoke(messages)

'Hello! How are you today?'

In [77]:
##Prompt template

from langchain_core.prompts import ChatPromptTemplate

system_message = "Translate the following text to {language}"
system_message1 = "who is the {profession} of "

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_message), ("user", "{text}")]
)

In [78]:
result = prompt_template.invoke({"language": "kannada", "text": "how are you doing"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following text to kannada'), HumanMessage(content='how are you doing')])

In [79]:
result.to_messages()

[SystemMessage(content='Translate the following text to kannada'),
 HumanMessage(content='how are you doing')]

In [80]:
prompt_template1 = ChatPromptTemplate.from_messages(
    [("system", system_message1), ("user", "{place}")]
)

result = prompt_template1.invoke({"profession": "prime minister", "place": "bhutan"})
result

ChatPromptValue(messages=[SystemMessage(content='who is the prime minister of '), HumanMessage(content='bhutan')])

In [81]:
chain_translate = prompt_template | model | parser
chain_translate.invoke({"language": "kannada", "text": "how are you doing"})

'ನೀವು ಹೇಗಿದ್ದೀರಿ?'

In [82]:
chain_profession = prompt_template1 | model | parser
chain_profession.invoke({"profession": "President", "place": "nepal"})

'The President of Nepal is Bidhya Devi Bhandari.'